<a href="https://colab.research.google.com/github/karensanb/rx/blob/main/organaMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install medmnist

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 2.3 MB/s eta 0:00:00


In [2]:
# Carga de librerías
import numpy as np, tensorflow as tf, matplotlib.pyplot as plt

# Carga de módulos
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
import tensorflow as tf

import torch
from torch.utils.data import DataLoader
import medmnist
from medmnist import OrganAMNIST

# Dataset

MNIST dataset

https://www.tensorflow.org/api_docs/python/tf/keras/datasets

In [4]:
#Carga MNIST

train_ds = OrganAMNIST(split='train', download=True, size=28)
val_ds   = OrganAMNIST(split='val', download=True, size=28)
test_ds  = OrganAMNIST(split='test', download=True, size=28)

#Obtener imágenes y etiquetas como arrays de NumPy
x_train, y_train = train_ds.imgs, train_ds.labels
x_val, y_val     = val_ds.imgs, val_ds.labels
x_test, y_test   = test_ds.imgs, test_ds.labels

#Revisar formas
print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_val:", x_val.shape, "y_val:", y_val.shape)
print("x_test:", x_test.shape, "y_test:", y_test.shape)

100%|██████████| 38.2M/38.2M [00:38<00:00, 993kB/s] 


x_train: (34561, 28, 28) y_train: (34561, 1)
x_val: (6491, 28, 28) y_val: (6491, 1)
x_test: (17778, 28, 28) y_test: (17778, 1)


Reshape a un tesor de 4 dimensiones: batch size, width, height, color channels (método reshape sobre datos (X) de entrenamiento)

In [5]:
#Normalización a [0,1]
x_train = x_train.astype("float32") / 255.0
x_val   = x_val.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

#Reshape
x_train_Mod = x_train.reshape(-1, 28*28)
x_val_Mod   = x_val.reshape(-1, 28*28)
x_test_Mod  = x_test.reshape(-1, 28*28)

#Verificamos las formas nuevas
print("x_train:", x_train.shape)
print("x_val:", x_val.shape)
print("x_test:", x_test.shape)

x_train: (34561, 28, 28)
x_val: (6491, 28, 28)
x_test: (17778, 28, 28)


Convertir etiquetas a one-hot-encoding (categóricas)

In [6]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Modelos (variación de Hiperparámetros)

In [7]:
#modelo 1
#Datos
x_train_Mod = x_train
y_train_Mod = y_train
x_val_Mod   = x_val
y_val_Mod   = y_val

#Hiperparámetros
batch_s, lr, num_epochs = 64, 0.0005, 20
optimizerf = tf.keras.optimizers.Adam(learning_rate=lr)

#Modelo
model = Sequential([
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dense(11, activation='softmax')
])

In [8]:
#Modelo 2
#Datos
x_train_Mod = x_train
y_train_Mod = y_train
x_val_Mod   = x_val
y_val_Mod   = y_val

#Hiperparámetros
batch_s, lr, num_epochs = 64, 0.0003, 25
optimizerf = tf.keras.optimizers.Adam(learning_rate=lr)
weight_decay = 1e-5

#Modelo
model = Sequential([
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    Dropout(0.3),
    Dense(11, activation='softmax')
])

In [9]:
#Modelo 3
#Datos
x_train_Mod = x_train
y_train_Mod = y_train
x_val_Mod   = x_val
y_val_Mod   = y_val

#Hiperparámetros
batch_s, lr, num_epochs = 128, 0.0001, 30
optimizerf = tf.keras.optimizers.Adam(learning_rate=lr)

#Modelo
model = Sequential([
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.4),
    Dense(512, activation='sigmoid'),
    Dense(256, activation='relu'),
    Dense(11, activation='softmax')
])

# Compilación y entrenamiento del modelo

In [10]:
model.compile(
  optimizer=optimizerf,
  loss= 'categorical_crossentropy',
  metrics=['CategoricalAccuracy']
)

# Entrenamiento

In [12]:
history = model.fit(
    x_train_Mod,
    y_train_Mod,
    epochs=num_epochs,
    batch_size=batch_s,
    validation_data=(x_test, y_test))

Epoch 1/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - CategoricalAccuracy: 0.9091 - loss: 0.2913 - val_CategoricalAccuracy: 0.7306 - val_loss: 0.8486
Epoch 2/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - CategoricalAccuracy: 0.9014 - loss: 0.3123 - val_CategoricalAccuracy: 0.7481 - val_loss: 0.7789
Epoch 3/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - CategoricalAccuracy: 0.9224 - loss: 0.2535 - val_CategoricalAccuracy: 0.7571 - val_loss: 0.7802
Epoch 4/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - CategoricalAccuracy: 0.9196 - loss: 0.2590 - val_CategoricalAccuracy: 0.7549 - val_loss: 0.7996
Epoch 5/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - CategoricalAccuracy: 0.9299 - loss: 0.2253 - val_CategoricalAccuracy: 0.7553 - val_loss: 0.8005
Epoch 6/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - CategoricalAccuracy: 0.9393 - loss: 0.2041 - val_CategoricalAccuracy: 0.7399 - val_loss: 0.8370
Epoch 7/30
271/271 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - CategoricalAccuracy: 0.9326 - lo